In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import pandas as pd

class GenreFeatsExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # Cette fonction ne nécessite pas d'apprentissage, donc nous retournons simplement l'objet lui-même
        return self

    def transform(self, X):
        # Créer une liste contenant tous les genres de films
        X_all_genres = X['genres'].str.split(', ', expand=True)
        # Convertir chaque genre en colonnes binaires
        X_all_genres = pd.get_dummies(X_all_genres.apply(lambda x: pd.Series(x).str.strip()), prefix='', prefix_sep='').groupby(level=0, axis=1).sum()
        
        # Renommer les colonnes binaires pour correspondre aux noms de colonnes du premier code
        X_all_genres.columns = ['genre_' + col.replace(' ', '_') for col in X_all_genres.columns]
        
        # Ajouter les colonnes binaires encodées en one-hot au DataFrame complet
        X_all = pd.concat([X, X_all_genres], axis=1)
        
        return X_all

    def get_feature_names_out(self, input_features=None):
        # get_feature_names_out doit retourner une liste de noms de colonnes pour les données transformées
        # Ici, nous renvoyons simplement toutes les colonnes du DataFrame après l'extraction des genres
        return self.transform(pd.DataFrame(input_features)).columns

# Fonction de pipeline de prétraitement
def preprocessing_pipeline(df):
    # Self Transformator
    # Création d'une instance de GenreFeatsExtractor
    gen = GenreFeatsExtractor()

    # Native transformator
    log_pipe = log_pipeline()
    cat_pipe = cat_pipeline()
    hash_pipe = hash_pipeline()

    # Simple modification/definition
    genre_feats = gen.get_feature_names_out(df)

    # Colonnes de caractéristiques
    features_columns = log_num_feats + cat_feats + genre_feats + count_feats

    # Transformation des données
    transformed_data = ColumnTransformer([
        ('gen', gen, ['genres']),
        ('log_num_feats', log_pipe, log_num_feats),
        ('cat_feats', cat_pipe, cat_feats),
        ('hash_feats', hash_pipe, genre_feats + count_feats)
    ], remainder='drop', n_jobs=-1).fit_transform(df) 

    transformed_df = pd.DataFrame(transformed_data, columns=features_columns)

    return transformed_df
